# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import os
from os import path
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import partial

import datetime as dt

repo_path = path.dirname(path.dirname(path.abspath("__file__")))
repo_path
sys.path.append(repo_path)

In [3]:
from unityagents import UnityEnvironment


In [4]:
from src.ac_agent import AgentDDPG, GaussianProcess, OUNoise
from src.utils import action_scaler_fn
from unity_reacher_utils  import train

In [5]:
RND_SEED = 123
EXP_NAME = 'ddpg:v02'
EXP_FOLDER = 'ddpg2'

In [6]:
# Problem
N_EPISODES = 5000
MAX_STEPS = 1000
SOLVED_AT = 30
GAMMA = 0.99  # Discount factor

# Noise
NOISE_MU = 0
NOISE_SIGMA = 0.1
NOISE_DECAY = 0.99975
NOISE_MIN_WEIGHT = 0.15

# Agent
ACT_HID_LAYERS = (256, 128)
CRIT_HID_LAYERS = (256, 128)
ACT_ADD_BN = (True)
CRIT_ADD_BN =(True)
GRAD_CLIP = (False, 1.)
BATCH_SIZE = 128
LEARNING_RATES = (1e-3, 1e-3)
WEIGHT_DECAY = (0, 0)
SOFT_UPD_PARAM = 1e-3
UPDATE_EVERY = 1
BUFFER_SIZE = int(1e6)
LEARN_EVERY = 20
LEARN_NUM = 10

In [7]:
action_scaler = partial(action_scaler_fn, lower=-1., upper=1.)
action_scaler(np.array(-0.5)), action_scaler(np.array(-3)), action_scaler(np.array(.5)), action_scaler(np.array(3)), 

(-0.5, -1.0, 0.5, 1.0)

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [8]:
env = UnityEnvironment(file_name='Reacher_Windows_x86_64/Reacher.exe')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [9]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [10]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [11]:
g_noise = GaussianProcess(action_size, RND_SEED, mu=NOISE_MU, sigma=NOISE_SIGMA)

In [12]:
ddpg_agent = AgentDDPG(
    state_size=state_size, action_size=action_size, gamma=GAMMA,
    actor_hidden_layers=ACT_HID_LAYERS, critic_hidden_layers=CRIT_HID_LAYERS,
    actor_add_bn=ACT_ADD_BN, critic_add_bn=CRIT_ADD_BN,
    grad_clipping=GRAD_CLIP,
    batch_size=BATCH_SIZE, learning_rates=LEARNING_RATES, weight_decay=WEIGHT_DECAY,
    soft_upd_param=SOFT_UPD_PARAM, update_every=UPDATE_EVERY, buffer_size=BUFFER_SIZE,
    noise=g_noise, learn_every=LEARN_EVERY, learn_num=LEARN_NUM,
    seed=RND_SEED, action_dtype='float')

In [13]:

path_ddpg_agent = os.path.join('models', EXP_FOLDER)

scores_ddpg = train(env, brain_name, ddpg_agent, n_episodes=N_EPISODES, max_t=MAX_STEPS, solved=SOLVED_AT,
                    action_scaler_fn=action_scaler,  add_noise=True, noise_decay=NOISE_DECAY, min_noise_weight=NOISE_MIN_WEIGHT,
                    model_save_path=path_ddpg_agent)

Episode [100/5000]	Average score: 1.63
Episode [200/5000]	Average score: 3.07
Episode [300/5000]	Average score: 3.29
Episode [400/5000]	Average score: 4.86
Episode [500/5000]	Average score: 5.01
Episode [600/5000]	Average score: 6.34
Episode [700/5000]	Average score: 7.48
Episode [800/5000]	Average score: 6.63
Episode [900/5000]	Average score: 7.79
Episode [1000/5000]	Average score: 9.78
Episode [1100/5000]	Average score: 10.33
Episode [1200/5000]	Average score: 11.66
Episode [1300/5000]	Average score: 13.86
Episode [1400/5000]	Average score: 12.95
Episode [1500/5000]	Average score: 15.08
Episode [1600/5000]	Average score: 15.77
Episode [1700/5000]	Average score: 17.53
Episode [1800/5000]	Average score: 19.77
Episode [1900/5000]	Average score: 25.64
Episode 1987	19s	Average Score: 30.14
Environment solved in 1887 episodes!	Average Score: 30.14


In [14]:
scores_ddpg['experiment'] = EXP_NAME

In [15]:
checkpoint_metadata = pd.Series(index=['N_episodes', 'gamma', 'actor_hidden_layers', 'critic_hidden_layers',
                'grad_clipping', 'batch_size', 'learning_rates',
                'soft_upd_param', 'update_every', 'buffer_size', 'noise', 'learn_every', 'learn_num', 'solved',
                'checkpoint_folder'],
         data = [len(scores_ddpg), GAMMA, ACT_HID_LAYERS, CRIT_HID_LAYERS, 
                 GRAD_CLIP, BATCH_SIZE, LEARNING_RATES,SOFT_UPD_PARAM, UPDATE_EVERY, BUFFER_SIZE, 'ou-noise', LEARN_EVERY, LEARN_NUM, False, EXP_FOLDER], name=f'experiment:{EXP_NAME}')
checkpoint_metadata

N_episodes                        1986
gamma                             0.99
actor_hidden_layers         (256, 128)
critic_hidden_layers        (256, 128)
grad_clipping             (False, 1.0)
batch_size                         128
learning_rates          (0.001, 0.001)
soft_upd_param                   0.001
update_every                         1
buffer_size                    1000000
noise                         ou-noise
learn_every                         20
learn_num                           10
solved                           False
checkpoint_folder                ddpg2
Name: experiment:ddpg:v02, dtype: object

In [16]:

experiment_dt = dt.datetime.strftime(dt.datetime.now(), "%Y%m%d%H%M%S")

In [17]:
checkpoint_metadata.to_json(f'models/experiments/hparams_{experiment_dt}.json')
scores_ddpg.to_csv(f'models/experiments/scores_{experiment_dt}.csv')